import the necessary library

In [2]:
import pandas as pd
# !pip install psycopg2
import pandas as pd
import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine


In [3]:
# Provide the file paths to the downloaded CSV files
borrower_csv_path = 'Borrower_Data - Sheet1.csv'
loan_csv_path = 'Loan_Data - Sheet1 - Loan_Data - Sheet1.csv'
payment_schedule_csv_path = 'Schedule_Data.xlsx - Sheet1.csv'
repayment_csv_path = 'Repayment_Data - Sheet1 - Repayment_Data - Sheet1 (1).csv'

# Read the CSV files into pandas DataFrames
borrower_df = pd.read_csv(borrower_csv_path)
loan_df = pd.read_csv(loan_csv_path)
payment_schedule_df = pd.read_csv(payment_schedule_csv_path)
repayment_df = pd.read_csv(repayment_csv_path)


In [4]:
borrower_df.head()

,Borrower_Id,State,City,zip code,borrower_credit_score
0,123fd35,dfgc,12olki,19473,4
1,123fd36,rtgio,89hfg,923475,a


In [5]:
loan_df.head()

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,1.007903e+05,01/31/2022
1,123fd36,9190i0-nbfb,02/01/2021,24,1.05,12394031,1000000,5.164180e+05,02/29/2023
2,123fd35,09u924rbwf,04/16/2021,36,1.05,45784937,1200000,1.271804e+06,04/30/2024
3,123fd36,3240-9rfwb,03/26/2021,12,1.05,1234890,132000,1.029075e+05,03/31/2022


In [6]:
payment_schedule_df.head()

,loan_id,schedule_id,Expected_payment_date,Expected_payment_amount
0,32u09wekjbfje,3434r409km123456,02/27/2021,100790.3333
1,32u09wekjbfje,3434r409km123457,03/27/2021,100790.3333
2,32u09wekjbfje,3434r409km123458,04/27/2021,100790.3333
3,32u09wekjbfje,3434r409km123459,05/27/2021,100790.3333
4,32u09wekjbfje,3434r409km123460,06/27/2021,100790.3333


In [7]:
repayment_df.head()

,loan_id(fk),payment_id(pk),Amount_paid,Date_paid
0,32u09wekjbfje,3434r409kmPAID123456,03/31/2021,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,03/31/2021,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,04/27/2021,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,05/27/2021,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,06/27/2021,100790.3333


In [8]:
repayment_df['Amount_paid'] = pd.to_datetime(repayment_df['Amount_paid'], errors='coerce')


In [9]:
repayment_df.head()

,loan_id(fk),payment_id(pk),Amount_paid,Date_paid
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333


In [10]:
loan_df['Date_of_release'] = pd.to_datetime(loan_df['Date_of_release'], errors='coerce')
loan_df['Maturity_date'] = pd.to_datetime(loan_df['Maturity_date'], errors='coerce')

In [11]:
loan_df.head()

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,32u09wekjbfje,2021-01-27,12,1.05,1209484,124993,1.007903e+05,2022-01-31
1,123fd36,9190i0-nbfb,2021-02-01,24,1.05,12394031,1000000,5.164180e+05,NaT
2,123fd35,09u924rbwf,2021-04-16,36,1.05,45784937,1200000,1.271804e+06,2024-04-30
3,123fd36,3240-9rfwb,2021-03-26,12,1.05,1234890,132000,1.029075e+05,2022-03-31


In [12]:
payment_schedule_df['Expected_payment_date'] = pd.to_datetime(payment_schedule_df['Expected_payment_date'], errors='coerce')

In [13]:
payment_schedule_df.head()

,loan_id,schedule_id,Expected_payment_date,Expected_payment_amount
0,32u09wekjbfje,3434r409km123456,2021-02-27,100790.3333
1,32u09wekjbfje,3434r409km123457,2021-03-27,100790.3333
2,32u09wekjbfje,3434r409km123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409km123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409km123460,2021-06-27,100790.3333


In [14]:
repayment_df.rename(columns={'Amount_paid':'payment_date'},inplace=True)
repayment_df.rename(columns={'Date_paid':'Amount_paid'},inplace=True)

In [15]:
repayment_df.head()

,loan_id(fk),payment_id(pk),payment_date,Amount_paid
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333


In [16]:
borrower_df.rename(columns={'zip code':'zip_code'},inplace=True)

In [17]:
borrower_df.head()

,Borrower_Id,State,City,zip_code,borrower_credit_score
0,123fd35,dfgc,12olki,19473,4
1,123fd36,rtgio,89hfg,923475,a


In [31]:
try:
    # Replace the connection details with your own
    connection = psycopg2.connect(
        host="localhost",
        database="autochek_db",
        user="auto",
        password="auto"
    )
    cursor = connection.cursor()
    print("Successfully connected to the PostgreSQL database")

except (Exception, Error) as e:
    print("Error while connecting to the PostgreSQL database:", e)


Successfully connected to the PostgreSQL database


In [19]:
loan_df.head()

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,32u09wekjbfje,2021-01-27,12,1.05,1209484,124993,1.007903e+05,2022-01-31
1,123fd36,9190i0-nbfb,2021-02-01,24,1.05,12394031,1000000,5.164180e+05,NaT
2,123fd35,09u924rbwf,2021-04-16,36,1.05,45784937,1200000,1.271804e+06,2024-04-30
3,123fd36,3240-9rfwb,2021-03-26,12,1.05,1234890,132000,1.029075e+05,2022-03-31


In [37]:
connection.rollback()

In [38]:
# SQL statements to create the tables
create_table_queries = [
    '''
    CREATE TABLE IF NOT EXISTS Borrower_table (
        Borrower_id INTEGER PRIMARY KEY,
        State TEXT,
        City TEXT,
        zip_code INTEGER,
        borrower_credit_score INTEGER
    )
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Loan_table (
        Borrower_id INTEGER REFERENCES Borrower_table (Borrower_id),
        loan_id INTEGER PRIMARY KEY,
        Date_of_release DATE,
        Term INTEGER,
        InterestRate FLOAT,
        LoanAmount FLOAT,
        Downpayment FLOAT,
        Payment_frequency TEXT,
        Maturity_date DATE
    )
    
    ''' ,
    '''
    CREATE TABLE IF NOT EXISTS Schedule_Table (
        loan_id INTEGER REFERENCES Loan_table (Loan_id),
        schedule_id INTEGER PRIMARY KEY,
        Expected_payment_date DATE,
        Expected_payment_amount FLOAT
    )
    
    ''' ,
    '''
    CREATE TABLE IF NOT EXISTS Repayment_Table (
        loan_id INTEGER REFERENCES Loan_table (Loan_id),
        payment_id INTEGER PRIMARY KEY,
        payment_date DATE,
        Amount_paid FLOAT
    )
    '''

]
try:
    # Execute the create table queries
    for query in create_table_queries:
        cursor.execute(query)
    connection.commit()
    print("Tables created successfully")

except (Exception, Error) as e:
    print("Error while creating tables:", e)

Tables created successfully


In [41]:
# Create a SQLAlchemy engine using the connection
engine = create_engine('postgresql+psycopg2://', creator=lambda: connection)

# Use the engine to load data into tables
try:
    
    borrower_df.to_sql('Borrower_table', con=engine, if_exists='replace', index=False)
    loan_df.to_sql('Loan_table', con=engine, if_exists='replace', index=False)
    payment_schedule_df.to_sql('Schedule_Table', con=engine, if_exists='replace', index=False)
    repayment_df.to_sql('Repayment_Table', con=engine, if_exists='replace', index=False)
    
    print("Data loaded into the tables successfully")

except (Exception, Error) as e:
    print("Error while loading data into tables:", e)

Data loaded into the tables successfully


In [50]:
pip install openpyxl


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 KB 491.8 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [60]:
import psycopg2

# Establish a connection to the PostgreSQL database
connection = psycopg2.connect(
    host="localhost",
    database="autochek_db",
    user="auto",
    password="auto"
)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Generate the SQL statement
sql_query = """
    SELECT
        b.City,
        b.zip_code,
        l.Payment_frequency,
        l.Maturity_date,
        (CURRENT_DATE - s.Expected_payment_date) AS current_days_past_due,
        MAX(s.Expected_payment_date) AS last_due_date,
        MAX(r.payment_date) AS last_repayment_date,
        SUM(
            CASE WHEN r.payment_date > s.Expected_payment_date
                THEN s.Expected_payment_amount
                ELSE 0
            END
        ) AS amount_at_risk
    FROM
        Borrower_table b
        INNER JOIN Loan_table l ON b.Borrower_id = l.Borrower_id
        INNER JOIN Schedule_Table s ON l.Loan_id = s.loan_id
        INNER JOIN Repayment_Table r ON l.Loan_id = r.loan_id
    GROUP BY
        b.City,
        b.zip_code,
        l.Payment_frequency,
        l.Maturity_date,
        (CURRENT_DATE - s.Expected_payment_date);
"""

try:
    # Execute the SQL query
    cursor.execute(sql_query)
    
    # Fetch all the results
    results = cursor.fetchall()
    
    # Create a DataFrame from the results
    df = pd.DataFrame(results, columns=["City", "Zip Code", "Payment Frequency", "Maturity Date",
                                        "Current Days Past Due", "Last Due Date", "Last Repayment Date",
                                        "Amount at Risk"])
    
    # Save the DataFrame to an Excel file
    output_file = "output_name.csv"
    df.to_csv(output_file, index=False)
    
    print("Query results saved to", output_file)

except (Exception, psycopg2.Error) as e:
    print("Error while executing the SQL query:", e)

finally:
    # Close the cursor and connection
    cursor.close()
    connection.close()

Query results saved to output_name.csv


In [54]:
import psycopg2

# Establish a connection to the PostgreSQL database
connection = psycopg2.connect(
    host="localhost",
    database="autochek_db",
    user="auto",
    password="auto"
)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Create a new table in the database to store the result
create_table_query2 = '''
CREATE TABLE IF NOT EXISTS output_name (
    City TEXT,
    "zip code" TEXT,
    "payment frequency" TEXT,
    "maturity date" DATE,
    "current_days_past_due" INTEGER,
    "last_due_date" DATE,
    "last_repayment_date" DATE,
    "amount_at_risk" FLOAT
)
'''
cursor.execute(create_table_query2)
connection.commit()

# Insert the DataFrame into the Result_Table
df_columns = df.columns.tolist()
columns = ",".join(df_columns)
values = "VALUES({})".format(",".join(["%s" for _ in df_columns]))
insert_query = "INSERT INTO output_name ({}) {}".format(columns, values)
cursor.executemany(insert_query, df.values.tolist())
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Query result saved to output_name in the database.")

Query result saved to output_name in the database.


In [56]:
import psycopg2

# Establish a connection to the database
connection = psycopg2.connect(
    host="localhost",
    database="autochek_db",
    user="auto",
    password="auto"
)

# Create a cursor object
cursor = connection.cursor()

# SQL query to calculate PAR days
query = """
SELECT
    l.Loan_id,
    s.Expected_payment_date,
    r.payment_date,
    (r.payment_date - s.Expected_payment_date) AS par_days
FROM
    Loan_table l
    INNER JOIN Schedule_Table s ON l.Loan_id = s.loan_id
    INNER JOIN Repayment_Table r ON l.Loan_id = r.loan_id
"""

try:
    # Execute the query
    cursor.execute(query)

    # Fetch all the results
    results = cursor.fetchall()

    # Print the results
    for row in results:
        loan_id = row[0]
        expected_payment_date = row[1]
        date_paid = row[2]
        par_days = row[3]
        print("Loan ID:", loan_id)
        print("Expected Payment Date:", expected_payment_date)
        print("Date Paid:", date_paid)
        print("PAR Days:", par_days)
        print()

except (Exception, psycopg2.Error) as e:
    print("Error while executing the SQL query:", e)

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
import pandas as pd
import psycopg2

# Establish a connection to the database
connection = psycopg2.connect(
    host="localhost",
    database="autochek_db",
    user="auto",
    password="auto"
)

# Create a cursor object
cursor = connection.cursor()

# SQL query to calculate PAR days
query = """
SELECT
    l.Loan_id,
    s.Expected_payment_date,
    r.payment_date,
    (r.payment_date - s.Expected_payment_date) AS par_days
FROM
    Loan_table l
    LEFT JOIN Schedule_Table s ON l.Loan_id = s.loan_id
    LEFT JOIN Repayment_Table r ON l.Loan_id = r.loan_id
"""

try:
    # Execute the query
    cursor.execute(query)

    # Fetch all the results
    results = cursor.fetchall()

    # Create a DataFrame from the results
    df = pd.DataFrame(results, columns=["Loan ID", "Expected Payment Date", "payment_date", "PAR Days"])

    # Save the DataFrame to an Excel file
    output_file = "output_name.xlsx"
    df.to_excel(output_file, index=False, encoding='utf-8-sig')
    print("Data saved successfully to", output_file)

except (Exception, psycopg2.Error) as e:
    print("Error while executing the SQL query:", e)

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
# Calculate PAR Days
repayment_df['Date_paid'] = pd.to_datetime(repayment_df['Date_paid']) #donethis
payment_schedule_df['Expected_payment_date'] = pd.to_datetime(payment_schedule_df['Expected_payment_date']) #donethis
repayment_df['PAR_Days'] = (repayment_df['Date_paid'] - payment_schedule_df['Expected_payment_date']).dt.days

# Calculate amount_at_risk
repayment_df['amount_at_risk'] = repayment_df.groupby('loan_id(fk)')['Amount_paid'].cumsum()

# Generate the SQL query using the desired columns
query = '''
SELECT borrower.Borrower_id, borrower.State, borrower.City, borrower.zip_code,
       loan.Loan_id, loan.Term, loan.InterestRate, loan.LoanAmount, loan.Downpayment,
       payment_schedule.Expected_payment_date, payment_schedule.Expected_payment_amount,
       loan_payment.PAR_Days, loan_payment.amount_at_risk
FROM Borrower_table AS borrower
JOIN Loan_table AS loan ON borrower.Borrower_id = loan.Borrower_id
JOIN Payment_Schedule AS payment_schedule ON loan.Loan_id = payment_schedule.loan_id
JOIN Loan_payment AS loan_payment ON loan.Loan_id = loan_payment.loan_id
ORDER BY borrower.Borrower_id, loan.Loan_id, payment_schedule.Expected_payment_date
'''

# Print the SQL query
print(query)
